In [1]:
import numpy as np
import pandas as pd

In [2]:
def choice_home(home_number):
    if home_number==1:
        return ["HHHHHHHHHHHHHHH",
                "HFFFFFFHFFHHHFH",
                "HFFHHFFHFFFFHFH",
                "HFFHHFFHFFFFHFH",
                "HFFFFFFHFFFFFFH",
                "HHHHHHFFFHHHHHH",
                "HFFFFHFFFHFFFFH",
                "HFFFFFFFFHFHHFH",
                "HFFFFHFFFFFFHFH",
                "HHHHHHFFFHFFFFH",
                "HFFFFHFFFHHHHHH",
                "HFFFFFFFFHFFFFH",
                "HHHHHHFFFHFFHFH",
                "HSFFFFFFFFFHHFH",
                "HHHHHHHHHHHHHHH"]

In [3]:
class Robot:
    np.random.seed(2)
    def __init__(self, home):
        self.gamma=0.9 #feedback oranı
        self.alpha=0.1 #öğrenme oranı
        self.epsilon=0.5 #
        self.max_episodes=10
        self.actions=['left','right','up','down']
        self.home = home
        self.y, self.x = self.create_robot_loc()
        self.map=self.create_map()
        self.update_map([self.y,self.x])
        self.frees=0
        self.Fpoint=0
        self.Hpoint=0
        self.samepoint=0
    def cal_home(self,l):
     H=l*4-4+63+4*(l-15)
     F=l*l-H
     constFail=25
     total=(F*self.Fpoint)+(H*self.Hpoint)+(constFail*self.samepoint)
     return total
    def change_points(self,f,h,same):
        self.Fpoint=f
        self.Hpoint=h
        self.samepoint=same

    def update_map(self,locs):
        self.map[locs[0],locs[1]]=1
    def create_map(self):
        return np.zeros([len(home),len(home[0])],int)
    def create_robot_loc(self):
        x=0
        y=0
        for line in home:
            for state in line:
                if (state == 'S'):
                    return y,x
                    break
                x+=1
            x=0
            y+=1
    def accuracy(self):
        freesCount=0
        for i in self.home:
            for k in i:
                if k=='F':
                    freesCount+=1
        return self.frees/freesCount*100
        
    def show_map(self):
        print(self.map)
        
    def get_loc(self):
        return self.y,self.x
        
    def set_loc(self,y,x):
        self.x=x
        self.y=y
        
    def act_feedback(self,act):
        y,x=self.get_loc()
        F=self.Fpoint
        H=self.Hpoint
        same=self.samepoint
        if act=='right':
            if home[y][x+1]!='H':
                if self.map[y][x+1]==0:
                    self.frees+=1
                    self.map[y][x+1]=1
                    self.set_loc(y,x+1)
                    R=F
                else:
                    R=same
            else:
                R=H
        if act=='left':
            if home[y][x-1]!='H':
                if self.map[y][x-1]==0:
                    self.frees+=1
                    self.map[y][x-1]=1
                    self.set_loc(y,x-1)
                    R=F
                else:
                    R=same
            else:
                R=H
        if act=='up':
            if home[y-1][x]!='H':
                if self.map[y-1][x]==0:
                    self.frees+=1
                    self.map[y-1][x]=1
                    self.set_loc(y-1,x)
                    R=F
                else:
                    R=same 
            else:
                R=H
        if act=='down':
            if home[y+1][x]!='H':
                if self.map[y+1][x]==0:
                    self.frees+=1
                    self.map[y+1][x]=1
                    self.set_loc(y+1,x)
                    R=F
                else:
                    R=same
            else:
                R=H
        return R
       
    def choise_act(self,Table):
        state_acts=Table.loc[self.get_loc()]
        if (np.random.uniform() > self.epsilon) or (state_acts==0).all():
            act_name = np.random.choice(self.actions)
        else:
            act_name=state_acts.idxmax()
        return act_name
        
    def qTable(self,states,act):
        q_table = pd.DataFrame(np.zeros((len(states), len(self.actions)))
                               ,index=pd.MultiIndex.from_tuples(states, names=['Row', 'Column']),columns=self.actions)
        return q_table

    def learn(self):
        table=self.qTable(states,self.actions)
        totalR=self.cal_home(len(self.home))
        for ep in range(self.max_episodes):
            self.set_loc(self.create_robot_loc()[0],self.create_robot_loc()[1])
            Done=True
            k=0
            while Done==True:
                oldState=self.get_loc()
                act=self.choise_act(table)
                R = self.act_feedback(act)
                k += R
                if totalR-5<k and k<totalR+5: #??
                    target= R
                    Done=False
                else :
                    target = R+self.gamma*table.loc[oldState].max()
                predict=table.loc[oldState,act]
                table.loc[oldState,act]+=self.alpha*(target-predict)
        return self.accuracy()  
    def learn_rewards(self,F,H,S):
        self.change_points(F,H,S) # F,H,same
        return self.learn()

In [4]:
home=choice_home(1)
states = [(i, j) for i in range(len(home)) for j in range(len(home[0]))]
robot1=Robot(home)

In [5]:
robot1.learn_rewards(1,-1,0) 

25.862068965517242